In [1]:
import pandas as pd 
import numpy as np  
import os 
import pandas as pd
#go to parent folder
os.chdir('../')

In [2]:
df = pd.read_excel(r'data\Results\EPFL_CO2_2023.xlsx')

In [8]:
df["CO2e kg"].sum()

9087845.865999918

In [4]:
from tableauhyperapi import HyperProcess, Connection, Telemetry, TableDefinition, SqlType, Inserter, TableName, CreateMode
import pandas as pd

def pre_process_source_df(source_columns_to_embed, source_df):
    source_df = source_df.dropna(subset=source_columns_to_embed,how = 'all')
        # Remove numbers from the specified columns
    for column in source_columns_to_embed:
        source_df[column] = source_df[column].str.replace(r'\d+', '', regex=True)
    return source_df

def update_dataframe_with_correction(source_df, corrected_df, key_column):
    source_df_copy = source_df.copy()
    corrected_df_copy = corrected_df.copy()
    # Reset index to ensure no duplicate labels interfere with the merge
    source_df_copy.reset_index(drop=True, inplace=True)
    corrected_df_copy.reset_index(drop=True, inplace=True)
    # Merging dataframes on 'Libellé article'
    merged_df = pd.merge(source_df_copy, corrected_df_copy, on=key_column, how='left', suffixes=('', '_corrected'))
    # List of columns to update
    columns_to_update = [col for col in corrected_df_copy.columns if col != key_column]
    # Update the columns in source_df with values from corrected_df
    for col in columns_to_update:
        # Update source_df_copy[col] with merged_df[f'{col}_corrected']
        source_df_copy[col] = merged_df.apply(
            lambda row: row[f'{col}_corrected'] if pd.notnull(row[f'{col}_corrected']) else None, axis=1
        )
    
    return source_df_copy

def get_sqltype(dtype):
    """Convert pandas dtype to Tableau Hyper SQLType."""
    if pd.api.types.is_integer_dtype(dtype):
        return SqlType.big_int()
    elif pd.api.types.is_float_dtype(dtype):
        return SqlType.double()
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return SqlType.timestamp()
    elif pd.api.types.is_bool_dtype(dtype):
        return SqlType.bool()
    else:
        return SqlType.text()

def df_to_hyper(df, output_path):
    """Export a pandas DataFrame to a Tableau Hyper file."""
    
    # Ensure all data is in the correct format before insertion
    receiver_data = []
    for _, row in df.iterrows():
        row_data = []
        for value in row:
            if pd.isnull(value):  # Handle NaN values
                row_data.append(None)
            else:
                row_data.append(value)
        receiver_data.append(row_data)
    
    # Define the table schema dynamically based on DataFrame columns
    table_definition = TableDefinition(
        table_name=TableName("Extract", "Extract"),
        columns=[
            TableDefinition.Column(col, get_sqltype(dtype)) for col, dtype in zip(df.columns, df.dtypes)
        ]
    )

    # Start the Hyper process and create the Hyper file
    with HyperProcess(telemetry=Telemetry.SEND_USAGE_DATA_TO_TABLEAU) as hyper:
        with Connection(endpoint=hyper.endpoint, database=output_path, create_mode=CreateMode.CREATE_AND_REPLACE) as connection:
            connection.catalog.create_schema("Extract")
            connection.catalog.create_table(table_definition)
            with Inserter(connection, table_definition) as inserter:
                inserter.add_rows(receiver_data)
                inserter.execute()

In [5]:
df_to_hyper(df, r'data\Results\EPFL_CO2_2023.hyper')

In [14]:
df_source= pd.read_excel(r'data\achats_EPFL\Extraction lignes de commandes CATALYSE 2023.xlsx')

In [15]:
df_source

,Code CDE,Date de commande,Référence article,Fournisseur,Libellé article,Unité Organisationnelle,Devise,PU commande,Qté commande,Montant total,...,Famille,ID de famille,fam_dom_order,fam_fam_order,fam_sfam_order,fam_ssfam_order,Familles Niveau 2,Entité de gestion,Statut,Type de DA
0,400366897,2023-01-01,10053999,839,"Delock DisplayPort — Mini DisplayPort (2 m, Di...",Laboratoire d'électronique et structures à l'é...,CHF,14.70,2.0,29.40,...,"Matériel électronique, accessoires et composants",158.0,"001000Operating equipment, supplies and tools",002000Electronic Components and Supplies,003000Electronic hardware and component parts ...,NaN,15.0,002000School of Engineering,En cours de commande,extern_catalog
1,400366897,2023-01-01,20952212,839,MSI GeForce RTX 3070 VENTUS 3X PLUS 8G OC LHR ...,Laboratoire d'électronique et structures à l'é...,CHF,603.90,1.0,603.90,...,"Matériel électronique, accessoires et composants",158.0,"001000Operating equipment, supplies and tools",002000Electronic Components and Supplies,003000Electronic hardware and component parts ...,NaN,15.0,002000School of Engineering,En cours de commande,extern_catalog
2,400366898,2023-01-01,737577-500MG,219,"2,3-Butadien-1-ol",Laboratoire de catalyse et synthèse asymétriques,CHF,80.00,1.0,80.00,...,Dérivés organiques et composés avec substitution,567.0,"001000Biological, chemical and gaseous consuma...",002000Chemicals including Bio Chemicals and Ga...,003000Compounds and mixtures,004000Organic derivatives and substituted comp...,3.0,002000School of Basic Sciences,En cours de commande,sciquest_catalog
3,400366899,2023-01-01,G6025-1VL,219,Geranylgeranyl pyrophosphate ammonium salt,Unité de la Prof. Oricchio,CHF,120.12,1.0,120.12,...,Produits biochimiques,568.0,"001000Biological, chemical and gaseous consuma...",002000Chemicals including Bio Chemicals and Ga...,003000Compounds and mixtures,004000Biochemicals,3.0,002000School of Life Sciences,En cours de commande,sciquest_catalog
4,400366900,2023-01-02,O3651340-1838330,547,eGFP_FW1;DNA-synthesis 3651340-1838330,Plateforme technologique Bertarelli de thérapi...,CHF,3.80,1.0,3.80,...,Fournitures et matériels de laboratoire,169.0,"001000Laboratory, measurement, observation and...",002000Laboratory and Measuring and Observing a...,003000Laboratory supplies and fixtures,NaN,18.0,002000School of Life Sciences,En cours de commande,extern_catalog
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147812,400450405,2023-12-31,19852,219,EasySepTM Mouse CD4+ T Cell Isolation Kit,Chaire Merck Serono en oncologie,EUR,601.35,1.0,601.35,...,"Kits de test, contrôles qualité, calibreurs et...",649.0,"001000Laboratory, measurement, observation and...",002000Laboratory and Measuring and Observing a...,003000Measuring and observing and testing inst...,004000Manual test kits and quality controls an...,18.0,002000School of Life Sciences,En cours de commande,sciquest_catalog
147813,400450406,2023-12-31,130-042-401,219,LS Separation columns,Chaire Merck Serono en oncologie,CHF,300.05,1.0,300.05,...,"Kits de test, contrôles qualité, calibreurs et...",649.0,"001000Laboratory, measurement, observation and...",002000Laboratory and Measuring and Observing a...,003000Measuring and observing and testing inst...,004000Manual test kits and quality controls an...,18.0,002000School of Life Sciences,En cours de commande,sciquest_catalog
147814,400450407,2023-12-31,74034,219,RNeasy Plus Micro Kit (50),Chaire Merck Serono en oncologie,CHF,499.59,1.0,499.59,...,"Éléments et kits d’extraction, de purification...",608.0,"001000Laboratory, measurement, observation and...",002000Laboratory and Measuring and Observing a...,003000Laboratory and scientific equipment,004000Nucleic acid extraction and purification...,18.0,002000School of Life Sciences,En cours de commande,sciquest_catalog
147815,400450407,2023-12-31,217684,219,miRNeasy Tissue/Cells Advanced Micro Kit (50),Chaire Merck Serono en oncologie,CHF,572.27,1.0,572.27,...,"Él